In [1]:
import pandas as pd, numpy as np

from IPython.display import HTML, Javascript
from labext.prelude import M

**Better display for pandas DataFrame**

In [2]:
M.DataTable.register()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
df = pd.DataFrame([
    ['name', 'phone', 'age'],
    ['john', '123-231-2321', 30],
    ['peter', '231-231-2321', 35],
])
df

,0,1,2
0,name,phone,age
1,john,123-231-2321,30
2,peter,231-231-2321,35


**Tippy:** shows pop-over of HTML elements

In [4]:
M.Tippy.register()

<IPython.core.display.Javascript object>

In [5]:
display(HTML('<button data-tippy-content="<b>Hello</b>, what are you doing?" data-tippy-allowHTML="true">Button</button>'))

In [6]:
M.Tippy.render()

<IPython.core.display.Javascript object>